<a href="https://colab.research.google.com/github/ShogoNoguchi/ABC_code/blob/main/%EF%BC%88%E9%80%B2%E6%8D%97%E3%83%8E%E3%83%BC%E3%83%88%E2%91%A0%EF%BC%89%EF%BD%9E%E5%A4%A7%E8%A6%8F%E6%A8%A1%E6%A7%8B%E9%80%A0%E3%81%AE%E7%89%B9%E5%AE%9A%E3%81%A8%E4%BF%AE%E6%AD%A3%E9%A0%85%E7%9B%AE%E3%81%AE%E7%99%BA%E8%A6%8B%EF%BD%9E.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

文責：群馬大学　野口翔伍

題：（進捗）～大規模構造の特定と修正項目の発見～

In [ ]:
# Google Driveをマウント
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


論文を熟読し、GitHubのREAD MEを読んで課題1つ目を処理する方法を調べた。
TUAB・TUEVどちらのデータセットを使っても良いと思ったが、どちらもTemple Universityへの要求方法が不明なため、質問済。
➡のちに、これらは使わず、SEED-Vを用いた感情分類をすることが判明。

リポジトリを[/content/drive/My Drive/LaBraM]へダウンロード済 12/2

In [ ]:
# Google Driveをマウント
from google.colab import drive
drive.mount('/content/drive')

# 保存先ディレクトリの設定
import os
drive_path = '/content/drive/My Drive/LaBraM'  # Drive内に保存するフォルダ名
if not os.path.exists(drive_path):
    os.makedirs(drive_path)

# リポジトリをクローン
repo_url = "https://github.com/935963004/LaBraM.git"  # GitHubのURL
%cd /content
!git clone {repo_url} labram_repo
!cp -r labram_repo/* "{drive_path}"

# ディレクトリ構造を表示
import os

def display_directory_structure(path, level=0):
    indent = "  " * level
    print(f"{indent}{os.path.basename(path)}/")
    if os.path.isdir(path):
        for item in os.listdir(path):
            display_directory_structure(os.path.join(path, item), level + 1)

print("ディレクトリ構造:")
display_directory_structure(drive_path)

# 推奨ファイルを特定するために 'checkpoints' フォルダの内容を表示
checkpoints_dir = os.path.join(drive_path, 'checkpoints')
if os.path.exists(checkpoints_dir):
    print("\n[checkpoints]フォルダの内容:")
    for file in os.listdir(checkpoints_dir):
        print(f"  {file}")
else:
    print("\n[checkpoints]フォルダが存在しません。")


Mounted at /content/drive
/content
Cloning into 'labram_repo'...
remote: Enumerating objects: 124, done.
remote: Counting objects: 100% (67/67), done.
remote: Compressing objects: 100% (25/25), done.
remote: Total 124 (delta 49), reused 45 (delta 42), pack-reused 57 (from 1)
Receiving objects: 100% (124/124), 167.16 MiB | 26.39 MiB/s, done.
Resolving deltas: 100% (55/55), done.
Updating files: 100% (27/27), done.
ディレクトリ構造:
LaBraM/
  checkpoints/
    labram-base.pth/
    vqnsp.pth/
  data_processor/
    data_preprocess.py/
    dataset.py/
  dataset_maker/
    make_TUAB.py/
    make_TUEV.py/
    make_h5dataset_for_pretrain.py/
    shock/
      __init__.py/
      utils/
        __init__.py/
        eegUtils.py/
        h5.py/
        ringBuffer.py/
  engine_for_finetuning.py/
  engine_for_pretraining.py/
  engine_for_vqnsp.py/
  labram.png/
  modeling_finetune.py/
  modeling_pretrain.py/
  modeling_vqnsp.py/
  norm_ema_quantizer.py/
  optim_factory.py/
  README.md/
  requirements.txt/
  r

大規模コードの木構造を特定 12/3

ファインチューニング時にはGiTHubの以下のコマンドにより
run_class_finetuning.pyが最初に起動される。

In [ ]:
OMP_NUM_THREADS=1 torchrun --nnodes=1 --nproc_per_node=8 run_class_finetuning.py \
        --output_dir ./checkpoints/finetune_tuab_base/ \
        --log_dir ./log/finetune_tuab_base \
        --model labram_base_patch200_200 \
        --finetune ./checkpoints/labram-base.pth \    #baseモデルの重み
        --weight_decay 0.05 \
        --batch_size 64 \
        --lr 5e-4 \
        --update_freq 1 \
        --warmup_epochs 3 \
        --epochs 30 \
        --layer_decay 0.65 \
        --drop_path 0.1 \
        --dist_eval \
        --save_ckpt_freq 5 \
        --disable_rel_pos_bias \
        --abs_pos_emb \
        --dataset TUAB \   #データセット名の指定
        --disable_qkv_bias \
        --seed 0

run_class_finetuning.pyから、以下の依存関係を特定した。

以下は、run_class_finetuning.py が依存しているコード群・クラス・関数を「文字による木構造」で示したものです（上から下に向かって依存が深まります）。また、run_class_finetuning.py 内の主要な関数呼び出しを、どのコード・関数へ依存しているかを付記します。実際には、importなどで多くの関数やクラスが呼ばれますが、特に本質的な依存先をまずは明確化します。

In [ ]:
run_class_finetuning.py
├─ get_args() : argparseによる引数パースを行い、ds_init（deepspeed初期化用）も取得
│
├─ main(args, ds_init)
│  ├─ utils.init_distributed_mode(args)
│  │   └─ torch.distributed (PyTorch公式)
│
│  ├─ (ds_initが有効な場合) utils.create_ds_config(args) : DeepSpeed用設定ファイル生成
│
│  ├─ get_dataset(args)
│  │   ├─ if args.dataset == 'TUAB': utils.prepare_TUAB_dataset("path/to/TUAB")
│  │   │    └─ utils.py 内のprepare_TUAB_dataset関数
│  │   ├─ if args.dataset == 'TUEV': utils.prepare_TUEV_dataset("path/to/TUEV")
│  │   │    └─ utils.py 内のprepare_TUEV_dataset関数
│  │   └─ 上記関数でDataLoader構築に必要なDatasetクラスを使用（TUABLoader/TUEVLoader, utils内）
│
│  ├─ get_models(args)
│  │   ├─ timm.models.create_model(args.model, ...)
│  │   │   ├─ modeling_finetune.py 内で@register_modelされたモデル(labram_base_patch200_200等)がtimmのレジストリ経由で作成
│  │   │   ├─ NeuralTransformerクラス (modeling_finetune.py)
│  │   │   └─ Attention / Block / PatchEmbed 等 (modeling_finetune.py 内で定義)
│
│  ├─ モデル読み込み・重みロード
│  │   └─ utils.load_state_dict(...)
│
│  ├─ Optimizer / Scheduler関連設定
│  │   └─ optim_factory.py の create_optimizer, get_parameter_groups 関数
│
│  ├─ 損失関数 (BCEWithLogitsLoss, CrossEntropyLoss等 : PyTorch公式)
│
│  ├─ utils.auto_load_model(args, model, ...)
│
│  ├─ トレーニングループ
│  │   ├─ for epoch in range(args.epochs):
│  │   │   ├─ train_one_epoch(...)
│  │   │   │   ├─ engine_for_finetuning.py の train_one_epoch関数
│  │   │   │   ├─ 内部でmodel(...)呼び出し (modeling_finetune.pyのNeuralTransformer.forward)
│  │   │   │   ├─ utils.get_metrics(...) for accuracyなど (utils.py)
│  │   │   └─ evaluate(...)
│  │   │       ├─ engine_for_finetuning.py の evaluate関数
│  │   │       └─ 内部で同様にmodel(...)呼び出し, utils.get_metrics(...)呼び出し
│
│  ├─ utils.save_model(...)でチェックポイント保存
│  └─ 全エポック終了後、時間計測などログ出力
│
└─ if __name__ == '__main__':
   ├─ opts, ds_init = get_args()
   └─ main(opts, ds_init)


さらに詳しくは以下のようになります。

In [ ]:
run_class_finetuning.py
├── argparse (標準ライブラリ)
├── datetime (標準ライブラリ)
├── numpy (外部ライブラリ)
├── time (標準ライブラリ)
├── torch (外部ライブラリ)
│   ├── torch.backends.cudnn
│   ├── torch.utils.data
│   └── torch.nn
├── json (標準ライブラリ)
├── os (標準ライブラリ)
├── pathlib.Path (標準ライブラリ)
├── collections.OrderedDict (標準ライブラリ)
├── timm (外部ライブラリ)
│   ├── timm.data.mixup.Mixup
│   ├── timm.models.create_model   #モデル構造を入れて、モデルを出力
│   ├── timm.loss.LabelSmoothingCrossEntropy
│   ├── timm.loss.SoftTargetCrossEntropy
│   └── timm.utils.ModelEma
├── optim_factory.py     #最適化関連
│   ├── create_optimizer
│   ├── get_parameter_groups
│   └── LayerDecayValueAssigner
├── engine_for_finetuning.py  #ファインチューニング用のエンジン
│   ├── train_one_epoch
│   └── evaluate
├── utils.py
│   ├── NativeScalerWithGradNormCount   #学習率スケーリング
│   ├── init_distributed_mode
│   ├── create_ds_config
│   ├── get_rank
│   ├── cosine_scheduler
│   ├── auto_load_model
│   ├── save_model
│   ├── is_main_process
│   ├── prepare_TUAB_dataset   #ここを変える必要がある。 utils内のprepare_oo_dataset系
│   ├── prepare_TUEV_dataset
│   ├── TensorboardLogger
│   └── load_state_dict
├── modeling_finetune.py    #ここを調べるとファインチューニングのモデル構造が分かる・変更できる。
│   └── モデルの定義
├── scipy.interpolate (外部ライブラリ)  #これはなんでインポートされてるか分からない...


12/4 run_class_finetuning.pyの実行時の関数呼び出し順序（木構造）

 ファインチューニング実行時のコード実行順序(ざっくりしたフロー)

README中のfinetune用コマンドは以下であった。

In [ ]:
OMP_NUM_THREADS=1 torchrun --nnodes=1 --nproc_per_node=8 run_class_finetuning.py \
        --output_dir ./checkpoints/finetune_tuab_base/ \
        --log_dir ./log/finetune_tuab_base \
        --model labram_base_patch200_200 \
        --finetune ./checkpoints/labram-base.pth \
        --weight_decay 0.05 \
        --batch_size 64 \
        --lr 5e-4 \
        --update_freq 1 \
        --warmup_epochs 3 \
        --epochs 30 \
        --layer_decay 0.65 \
        --drop_path 0.1 \
        --dist_eval \
        --save_ckpt_freq 5 \
        --disable_rel_pos_bias \
        --abs_pos_emb \
        --dataset TUAB \
        --disable_qkv_bias \
        --seed 0

上記コマンドを動かしたとき、主な処理フロー（時間的順序）は以下の通り。

In [ ]:
1. run_class_finetuning.py が実行される
   ├─ get_args()で引数をパース
   ├─ ds_initが必要ならDeepSpeed設定
   └─ main(args, ds_init)を呼び出す

2. main(args, ds_init):
   ├─ utils.init_distributed_mode(args)で分散学習設定 (MPI or env var)
   ├─ ds_initがある場合：utils.create_ds_config(args)でDeepSpeed用ファイル生成
   ├─ get_dataset(args)呼び出し
   │  ├─ args.datasetが'TUAB'の場合: utils.prepare_TUAB_dataset(...)実行
   │  ├─ 得られたtrain/val/test datasetをDataLoader化
   ├─ get_models(args)でモデル構築
   │  ├─ timm.create_model(args.model)がmodeling_finetune.py内のNeuralTransformerインスタンス化
   ├─ 事前学習済みチェックポイント(--finetune指定)をutils.load_state_dictでモデルにロード
   ├─ optim_factory.create_optimizer(...)でOptimizer生成
   ├─ 損失関数、学習率スケジューラ、weight decayスケジューラなど設定
   ├─ utils.auto_load_model(...)で中断再開処理（過去checkpointがあればロード）

3. トレーニング開始
   ├─ for epoch in range(args.epochs):
   │  ├─ train_one_epoch(...) (engine_for_finetuning.py)
   │  │  ├─ DataLoaderからバッチ取得
   │  │  ├─ model(...) forward計算
   │  │  ├─ 損失逆伝播、optimizer.step()
   │  │  └─ メトリクス計算 (utils.get_metrics)
   │  ├─ evaluate(...)で検証データ推論/精度計算
   │  ├─ 条件に応じてutils.save_model()でcheckpoint保存
   │  └─ ログWriter(Tensorboard)更新

4. 全epoch終了後、終了ログ出力しプログラム終了


より詳しくは以下の通り。

In [ ]:
run_class_finetuning.py
└── main(args, ds_init)
    ├── utils.init_distributed_mode(args)
    ├── デバイスとシードの設定
    ├── cudnn.benchmark = True
    ├── dataset_train, dataset_test, dataset_val, ch_names, metrics = get_dataset(args) #データセットの取得
    │   └── prepare_TUAB_dataset または prepare_TUEV_dataset（utils.py内）
    ├── サンプラーとデータローダーの設定
    │   ├── torch.utils.data.DistributedSampler（トレーニング用）
    │   ├── torch.utils.data.SequentialSampler（検証・テスト用）
    │   └── torch.utils.data.DataLoader（データローダー作成）
    ├── ロガーの設定（TensorboardLogger）
    ├── model = get_models(args)
    │   └── timm.models.create_model による。create_modelの中で args.modelが渡されてmodeling_finetune.py の中のモデル（labram_base_patch200_200）構造を取得する。
    ├── モデルの初期化と読み込み   #モデルの重みがここで読まれる。
    │   ├── 事前学習済みモデルのロード（args.finetune が指定されている場合）
    │   ├── モデルの重みの調整（必要に応じて）
    │   └── モデルをデバイスに転送（model.to(device)）
    ├── モデルEMAの設定（必要に応じて）
    ├── DistributedDataParallel の設定（分散学習の場合）
    ├── オプティマイザとスケジューラの設定
    │   ├── optimizer = create_optimizer(...)
    │   ├── loss_scaler = NativeScaler()
    │   ├── lr_schedule_values = utils.cosine_scheduler(...)
    │   └── wd_schedule_values = utils.cosine_scheduler(...)
    ├── 損失関数の設定（criterion）
    ├── utils.auto_load_model(...)（チェックポイントの自動ロード）
    ├── トレーニングループ開始（エポックごとに繰り返し）
    │   ├── data_loader_train.sampler.set_epoch(epoch)
    │   ├── train_stats = train_one_epoch(...)
    │   │   ├── モデルをトレーニングモードに設定（model.train()）
    │   │   ├── バッチごとのトレーニングループ
    │   │   │   ├── 入力データとラベルを取得
    │   │   │   ├── optimizer.zero_grad()
    │   │   │   ├── 出力の計算（outputs = model(inputs)）
    │   │   │   ├── 損失の計算（loss = criterion(outputs, targets)）
    │   │   │   ├── loss_scaler(loss, optimizer, ...)
    │   │   │   └── 学習率と重み減衰の更新
    │   │   └── 統計情報の収集
    │   ├── モデルの保存（必要に応じて）
    │   ├── 検証（data_loader_val がある場合）
    │   │   ├── val_stats = evaluate(...)
    │   │   │   ├── モデルを評価モードに設定（model.eval()）
    │   │   │   ├── バッチごとの評価ループ
    │   │   │   │   ├── 入力データとラベルを取得
    │   │   │   │   ├── 出力の計算（outputs = model(inputs)）
    │   │   │   │   ├── 損失の計算（loss = criterion(outputs, targets)）
    │   │   │   │   └── 統計情報の収集（metrics の計算）
    │   │   └── 検証結果の表示とログ記録
    │   ├── テスト（data_loader_test がある場合）
    │   │   ├── test_stats = evaluate(...)
    │   │   └── テスト結果の表示とログ記録
    │   ├── 最良モデルの保存（必要に応じて）
    │   └── エポックごとのログの保存
    └── トレーニング時間の計測と表示


分類対象の数を変えたいときはモデルのヘッドを変えないといけない、まずモデル構造の変えかたを調べよう。


ーーーモデル構造についてーーー

modeling_finetune.pyの中でtimm の @register_model デコレーターを用いて、カスタムモデル(labram_base_patch200_200)が登録されています。

args.model = 'labram_base_patch200_200'となってる影響で、

get_model関数＝create_modelの引数をargs.model とすると。labram_base_patch200_200 構造をtimmのcreate_modelが作ってくれる仕組み。
以下modeling_finetuneで独自モデルが登録されてる様子。

（NeuralTransformerはLaBramの本体的クラス）

In [ ]:
@register_model
def labram_base_patch200_200(pretrained=False, **kwargs):
    model = NeuralTransformer(
        patch_size=200, embed_dim=200, depth=12, num_heads=10, mlp_ratio=4,
        norm_layer=partial(nn.LayerNorm, eps=1e-6), **kwargs)
    return model




ーーーーーーーーーーーーーー

run_class_finetuning.pyの
main関数内でモデルは以下のようにロードされる。
以下のコードは、モデルの構造（modeling_finetune.py で定義された構造）に合わせてチェックポイントからロードした重みを調整する仕組みになっている。
特に、ロードした重みの形状が現在のモデル構造と一致しない場合には、該当部分の重みを削除して適用されないようにしている点が特徴。

つまり、元のLaBramの自己教師モデルの重みのヘッド部分を強制的に消して、クラス分類へ適用するようになってる。


In [ ]:
model = get_models(args)  #モデル生成。 get_modelsはrun_class_finetuning中で定義

    patch_size = model.patch_size
    print("Patch size = %s" % str(patch_size))
    args.window_size = (1, args.input_size // patch_size)
    args.patch_size = patch_size

    if args.finetune:
        if args.finetune.startswith('https'):
            checkpoint = torch.hub.load_state_dict_from_url(
                args.finetune, map_location='cpu', check_hash=True)
        else:
            checkpoint = torch.load(args.finetune, map_location='cpu')

        print("Load ckpt from %s" % args.finetune)
        checkpoint_model = None
        for model_key in args.model_key.split('|'):
            if model_key in checkpoint:
                checkpoint_model = checkpoint[model_key]
                print("Load state_dict by model_key = %s" % model_key)
                break
        if checkpoint_model is None:
            checkpoint_model = checkpoint
        if (checkpoint_model is not None) and (args.model_filter_name != ''):
            all_keys = list(checkpoint_model.keys())
            new_dict = OrderedDict()
            for key in all_keys:
                if key.startswith('student.'):
                    new_dict[key[8:]] = checkpoint_model[key]
                else:
                    pass
            checkpoint_model = new_dict

        state_dict = model.state_dict()
        for k in ['head.weight', 'head.bias']:
            if k in checkpoint_model and checkpoint_model[k].shape != state_dict[k].shape:
                print(f"Removing key {k} from pretrained checkpoint")
                del checkpoint_model[k]

        all_keys = list(checkpoint_model.keys())
        for key in all_keys:
            if "relative_position_index" in key:
                checkpoint_model.pop(key)

        utils.load_state_dict(model, checkpoint_model, prefix=args.model_prefix)


In [ ]:
# get_models


def get_models(args):
  #create_model は、PyTorch Image Models (timm) ライブラリが提供するユーティリティ関数で、モデルを簡単に生成するための関数です。
    model = create_model(
        args.model,
        pretrained=False,
        num_classes=args.nb_classes,  #分類クラス数
#ドロップアウト率
        drop_rate=args.drop,
        drop_path_rate=args.drop_path,
        attn_drop_rate=args.attn_drop_rate,
        drop_block_rate=None,

        use_mean_pooling=args.use_mean_pooling,
        init_scale=args.init_scale,
        use_rel_pos_bias=args.rel_pos_bias,
        use_abs_pos_emb=args.abs_pos_emb,
        init_values=args.layer_scale_init_value,
        qkv_bias=args.qkv_bias,
    )

    return model


以上から、私がもし分類数を変えたければ、args.nb_classesの値（後述するget_dataset(args)内で指定）を変えればいいことが分かる。

もし分類ヘッドが希望するものに対応してなければ、modeling_finetune.py内の分類ヘッドを変えればいいだろう。

12/5

ここまで分かったことを一旦まとめて明日につなげる。

まず、公式手順では、以下がファインチューニングに必須：

run_class_finetuning.py：ファインチューニング実行スクリプト

engine_for_finetuning.py：finetuning時のtrain/evalループ

modeling_finetune.py：fine-tune用モデル定義
optim_factory.py：最適化関連

utils.py：データ準備、分散初期化、評価指標など

データセット生成スクリプト(make_TUAB.pyやmake_TUEV.pyはサンプル)はデータごとに必要だが公式はTUAB/TUEVを例示している


逆に、ファインチューニングそのものに不要な.pyは、事前学習やTokenizer学習に関わるもの、具体的には：

run_vqnsp_training.py：Neural Tokenizer(VQ)の学習用

run_labram_pretraining.py：LaBraM事前学習用

engine_for_pretraining.py：事前学習ループ用

engine_for_vqnsp.py：VQ tokenizer学習ループ用

modeling_pretrain.py：事前学習用モデルアーキテクチャ定義

modeling_vqnsp.py：Neural tokenizerモデル定義

norm_ema_quantizer.py：Vector-Quantized表現学習用コードブック管理用

dataset_maker/make_h5dataset_for_pretrain.py：事前学習用HDF5データ作成用スクリプト

(TUAB,TUEV以外を使うなら)make_TUAB.py, make_TUEV.pyは厳密には不要（自分のデータセット用の類似スクリプトを作成すればよい）

これらは事前学習やトークナイザ学習に必要で、すでにlabram-base.pthなどの事前学習済みモデルがある前提でfine-tuneするだけなら不要となる。




【任意データセット・任意電極位置でファインチューニングするための戦略と手順】


①データフォーマットの統一

TUAB/TUEV用にはmake_TUAB.pyやmake_TUEV.pyで前処理・HDF5 or pkl形式への変換を行っている。

→ SEED-Vデータセットも同様に"HDF5"または同様のpklフォーマットへ変換するスクリプトmake_SEEDV.py（自作）を作る。

②run_class_finetuning.pyでのデータセット読込処理対応:

get_dataset(args)関数内でif args.dataset == 'SEEDV': ...のような条件分岐を追加し、prepare_SEEDV_dataset()関数（自作）を呼ぶ。
utils.pyにprepare_SEEDV_dataset関数を定義し、SEED-V用のデータローダ（Datasetクラス）を返せるようにする。またはdataset_makerフォルダ内でTUAB/TUEVを参考にSEED-V用コードを書く。
SEED-Vは5クラス分類なのでargs.nb_classes = 5とするか、コード内で自動設定する。

③電極チャンネルセットへの対応:

現状get_input_chans(ch_names)関数で10-20 systemベースのチャネルインデックスを取得している。
任意の電極位置セットを用いる場合は、この部分をカスタマイズする必要がある。例えば、SEED-V特有のチャネル命名に合わせてutils.get_input_chans(ch_names)を修正するか、追加のパラメータを与え、マッピングルールを変更する。

もしくは、modeling_finetune.py内のNeuralTransformerでパッチ分割前にチャネル数が固定想定されているなら、入力チャネル数に合わせてin_chansやout_chans設定を変更（Baseモデルは特定チャネル数を仮定しているが、必要ならモデル定義を拡張し、柔軟に対応する）。

④モデル出力層の次元変更:

NeuralTransformerのself.head = nn.Linear(embed_dim, num_classes)で出力クラス数を決める。
run_class_finetuning.py内でargs.nb_classesを新たなデータセットのクラス数に合わせて設定し、モデル生成時にnum_classes=args.nb_classesを渡せばOK。

⑤評価メトリクスの構築（これは必要か後でもう一回検討）

SEED-Vでの実験では、Balanced Accuracyの代わりにAccuracyが使われているので、これを構築する。

⑤Colabでの実行:

!pip install -r requirements.txtなどで依存パッケージをインストール。
Google Drive上にリポジトリとデータセットを配置。
%cd /content/drive/MyDrive/LaBram/でディレクトリ移動して!python run_class_finetuning.py ...で実行。
分散は不要な場合、--nproc_per_node=1などで単GPU実行に変更可能。
このように手順を踏めば、任意のEEGデータセット・電極セットでfine-tuningできる。


12/6 次に、今日は異なるデータセットに対して処理を行うために、データセットを処理する関数について調べよう。まだSEED-V等データセットは入手できていないが、処理関数の中を調べることで、EDA・前処理用関数作成を加速できる。

まずはデータセットが処理される関数をたどっていく（源流を探索する）

上述したように、run_class_finetuning関数内のmain関数で、データセットは

In [ ]:
dataset_train, dataset_test, dataset_val, ch_names, metrics = get_dataset(args)

このように得られている。 get_dataset(args)は同コード内で以下のように定義されている。

In [ ]:
#この関数内に、get_dataset(args)関数内でif args.dataset == 'SEEDV': ...のような条件分岐を追加し、prepare_SEEDV_dataset()関数（自作）を呼びたい。


def get_dataset(args):
    if args.dataset == 'TUAB':
        train_dataset, test_dataset, val_dataset = utils.prepare_TUAB_dataset("path/to/TUAB")

#utils.prepare_TUAB_dataset という記述から、prepare_TUAB_dataset 関数はutils モジュール（またはスクリプト）内に定義されていることを示しています。

      #チャンネル名の指定
        ch_names = ['EEG FP1', 'EEG FP2-REF', 'EEG F3-REF', 'EEG F4-REF', 'EEG C3-REF', 'EEG C4-REF', 'EEG P3-REF', 'EEG P4-REF', 'EEG O1-REF', 'EEG O2-REF', 'EEG F7-REF', \
                    'EEG F8-REF', 'EEG T3-REF', 'EEG T4-REF', 'EEG T5-REF', 'EEG T6-REF', 'EEG A1-REF', 'EEG A2-REF', 'EEG FZ-REF', 'EEG CZ-REF', 'EEG PZ-REF', 'EEG T1-REF', 'EEG T2-REF']



        ch_names = [name.split(' ')[-1].split('-')[0] for name in ch_names] #各チャンネル名を加工し、余分な情報を削除して簡潔な名前（例: 'FP1', 'FP2'）に変換
        args.nb_classes = 1    #分類タスクのクラス数設定
        metrics = ["pr_auc", "roc_auc", "accuracy", "balanced_accuracy"]
    elif args.dataset == 'TUEV':       #TUABじゃなければTUEV
        train_dataset, test_dataset, val_dataset = utils.prepare_TUEV_dataset("path/to/TUEV")
        ch_names = ['EEG FP1-REF', 'EEG FP2-REF', 'EEG F3-REF', 'EEG F4-REF', 'EEG C3-REF', 'EEG C4-REF', 'EEG P3-REF', 'EEG P4-REF', 'EEG O1-REF', 'EEG O2-REF', 'EEG F7-REF', \
                    'EEG F8-REF', 'EEG T3-REF', 'EEG T4-REF', 'EEG T5-REF', 'EEG T6-REF', 'EEG A1-REF', 'EEG A2-REF', 'EEG FZ-REF', 'EEG CZ-REF', 'EEG PZ-REF', 'EEG T1-REF', 'EEG T2-REF']
        ch_names = [name.split(' ')[-1].split('-')[0] for name in ch_names]
        args.nb_classes = 6
        metrics = ["accuracy", "balanced_accuracy", "cohen_kappa", "f1_weighted"]
    return train_dataset, test_dataset, val_dataset, ch_names, metrics



ここで、データセットによって変わる「args.nb_classesの値」に注意したい。これは、のちにmain関数内で
args.nb_classes == 1 の場合:
BCEWithLogitsLoss（バイナリ分類用損失関数）が使用される。
args.nb_classes > 1 の場合:
ラベルスムージングを伴う CrossEntropyLoss または通常の CrossEntropyLoss（多クラス分類用損失関数）が選択される。

という風に影響する。

次に、データセット取得元となっているutilsのutils.prepare_TUEV_dataset("path/to/TUEV")を見る。
（我々がやりたいのは多値分類のため）

In [ ]:
"以下utils.pyより。
"私はこれを参考にして、utils.pyにprepare_SEEDV_dataset関数を定義し、
"SEED-V用のデータローダ（Datasetクラス）を返せるようにする。またはdataset_makerフォルダ内でTUAB/TUEVを参考にSEED-V用コードを書く。

def prepare_TUEV_dataset(root):
    # set random seed
    seed = 4523  #再現性を確保するために、ランダムシードを設定
    np.random.seed(seed)

    train_files = os.listdir(os.path.join(root, "processed_train"))
    val_files = os.listdir(os.path.join(root, "processed_eval"))
    test_files = os.listdir(os.path.join(root, "processed_test"))

    # prepare training and test data loader
    #データセット固有のデータローダを使っている。
    train_dataset = TUEVLoader(
        os.path.join(
            root, "processed_train"), train_files
    )
    test_dataset = TUEVLoader(
        os.path.join(
            root, "processed_test"), test_files
    )
    val_dataset = TUEVLoader(
        os.path.join(
            root, "processed_eval"), val_files
    )
    print(len(train_files), len(val_files), len(test_files))
    return train_dataset, test_dataset, val_dataset



SEED-V固有のデータローダも作る必要がある。
以下TUEVLoader.

In [ ]:

"utils.py
class TUEVLoader(torch.utils.data.Dataset):
    def __init__(self, root, files, sampling_rate=200):
        self.root = root
        self.files = files
        self.default_rate = 200
        self.sampling_rate = sampling_rate

    def __len__(self):
        return len(self.files)

    def __getitem__(self, index):
        sample = pickle.load(open(os.path.join(self.root, self.files[index]), "rb"))
        X = sample["signal"]
        if self.sampling_rate != self.default_rate:
            X = resample(X, 5 * self.sampling_rate, axis=-1)
        Y = int(sample["label"][0] - 1)
        X = torch.FloatTensor(X)
        return X, Y



12/6
今日やる事：
SEED-Vデータセットも同様に"HDF5"または同様のpklフォーマットへ変換するスクリプトmake_SEEDV.py（自作）を作る。

ここからは一旦このノートの記録を停止する。